In [30]:
import dask.dataframe as dd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from scikeras.wrappers import KerasClassifier
from dask_ml.wrappers import Incremental
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split

In [2]:
data = dd.read_csv('../data/intravehicle.csv')

In [3]:
y_data = data.pop('4')
x_data = data

In [4]:
train_x,test_x,train_y,test_y = train_test_split(x_data,y_data,test_size=0.2,random_state=0,shuffle=True)

In [6]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [7]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [32]:
model = Sequential()
model.add(Input(shape=(len(x_data.columns),1)))
model.add(LSTM(100))
model.add(Dense(4,activation='softmax'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 100)               40800     
                                                                 
 dense_6 (Dense)             (None, 4)                 404       
                                                                 
Total params: 41,204
Trainable params: 41,204
Non-trainable params: 0
_________________________________________________________________


In [33]:
classifier = KerasClassifier(model=model, random_state=0, optimizer='adam', epochs=10)

In [34]:
inc = Incremental(classifier,scoring='accuracy')

In [ ]:
inc.fit(train_x,train_y,classes=['Normal','DoS','Fuzzy','Spoof'])

In [ ]:
inc.score(test_x,test_y)

0.8617261916613721